# 只更新了pre word count这个类 更新时候仅需要更新这个即可

In [2]:
import json

import pickle
import json
import time
import os
import re
import pandas as pd
import numpy as np
import pickle as pkl
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
#导入PCA算法库
from textrank4zh import TextRank4Keyword, TextRank4Sentence
from sklearn import ensemble
import math
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn.externals import joblib
from elasticsearch import Elasticsearch

In [3]:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class esPaperRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True

    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esPaperRetrieval, cls).__new__(cls)
        return cls._instance

    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esPaperRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'paper-detail-index'

    def do_search(self, titleQuery, kwordQuery, summaryQuery, pubQuery, fromDate, toDate, volume):
        '''
        do_search方法执行具体检索过程
        titleQuery 为对应标题检索词
        kwordQuery 为对应关键字检索词
        summaryQuery 为对应摘要检索词
        上述三个词为or模式 可以出现 可以不出现
        pubQuery 为对应文摘关键词 这个关键词必定匹配
        fromDate 为检索字段起始日期 toDate为终结日期 日期格式 yyyy-mm-dd
        volume为每次检索返回的数目
        '''
        queryBody = {
            "query": {
                "bool": {
                    "should": [
                        {
                            "term": {
                                "P_Title": titleQuery
                            }
                        },
                        {
                            "term": {
                                "P_Keyword": kwordQuery
                            }
                        },
                        {
                            "term": {
                                "P_Summary": summaryQuery
                            }
                        },
                        {
                            "term": {
                                "P_Publication.keyword": {"value": pubQuery}
                            }
                        }
                    ],
                    "filter": [
                        {
                            "range": {
                                "P_year": {
                                    "gt": fromDate,
                                    "lt": toDate
                                }
                            }
                        },
                    ]
                }
            },
            "from": 0,
            "size": volume,
            "sort": [],
            "aggs": {}
        }
        #         print(queryBody)
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'P_ID, P_Title, P_Author, P_Publication, P_Organ, P_year, P_Keyword, \
    P_Summary, P_Keyword_seg, P_Title_seg,\
    P_Summary_seg, P_URL, P_Fields, P_Fields_two,P_References, P_Pagecount, P_Page, P_Language,\
    P_Download_num, P_Citation_num,P_Vector,P_Volume, P_Issue,P_Issn,P_Isbn, P_Doi,\
    P_Red1, P_Red2, P_Red3, P_Red4, P_Red5'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs

    def Retrieval(self, titleQuery, kwordQuery, summaryQuery, pubQuery, fromDate, toDate, volume):
        result = self.do_search(titleQuery, kwordQuery, summaryQuery, pubQuery, fromDate, toDate, volume)
        docs = self.format_search(result)
        return docs

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [4]:
class author_search():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True

    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(author_search, cls).__new__(cls)
        return cls._instance

    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(author_search, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'paper-detail-index'

    def do_search(self, nameQuery, organQuery, volume):
        '''
        do_search方法执行具体检索过程
        titleQuery 为对应标题检索词
        kwordQuery 为对应关键字检索词
        summaryQuery 为对应摘要检索词
        上述三个词为or模式 可以出现 可以不出现
        pubQuery 为对应文摘关键词 这个关键词必定匹配
        fromDate 为检索字段起始日期 toDate为终结日期 日期格式 yyyy-mm-dd
        volume为每次检索返回的数目
        '''
        queryBody = {
            "query": {
                "match_phrase": {
                    "P_Author": {
                        "query": nameQuery,
                        "slop": 2
                    }
                }},
            "from": 0,
            "sort": [],
            "size": volume,
            "aggs": {}
        }
        self.name = nameQuery
        self.organ = organQuery
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'P_Author,P_Title,  P_Publication, P_Organ, P_Keyword, P_Title_seg,P_Red3,P_Keyword,P_Red1,P_Red2'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs

    def Retrieval(self, nameQuery, organQuery, volume):
        result = self.do_search(nameQuery, organQuery, volume)
        self.docs = self.format_search(result)
        return self.result()

    def result(self, ):
        title = []
        first_authors = []
        authors = []
        parter_authors = []
        organ = []
        keywords = []
        jounal = []
        first_organ = []
        for i in range(len(self.docs)):
            title.append(self.docs[i]['P_Title'])
            authors.append(self.docs[i]['P_Author'])
            first_authors.append(authors[i].split(';')[0])
            parter_authors.append(authors[i].split(';')[1:])
            organ.append(self.docs[i]['P_Organ'])
            keywords.append(self.docs[i]['P_Keyword'])
            jounal.append(self.docs[i]['P_Publication'])
            first_organ.append(organ[i].split(';')[0])
            df = pd.DataFrame({
                'title': title,
                'first_authors': first_authors,
                'authors': authors,
                'parter_authors': parter_authors,
                'organ': organ,
                'keywords': keywords,
                'Journal': jounal,
                'first_authors_organ': first_organ
            })
        self.papers_author = df.drop_duplicates(subset=['title'], keep='first')
        return self.search()

    def savepickle(self, name, data):
        output = open('%s.pkl' % (name), 'wb')
        pickle.dump(data, output)
        output.close()

    def loadpickle(self, name):
        pkl_file = open('%s.pkl' % (name), 'rb')
        data1 = pickle.load(pkl_file)
        return data1

    def commonElement(self, a, b):
        commonEle = [val for val in a if val in b]
        return commonEle

    def all_list(self, list1):
        result = {}
        for i in set(list1):
            result[i] = list1.count(i)
        return result

    def search(self, ):
        self.authors_dict = {}
        self.author_dict = {}
        quchong_list = []
        quchong_list2 = []
        self.author_index = 0
        j = 0
        aa = "没有找到相匹配的学者"
        self.author_list = list(self.papers_author['first_authors'])
        author_organ_list = list(self.papers_author['first_authors_organ'])
        self.index_list = [idx for idx, i in enumerate(self.author_list) if i == self.name]
        if len(self.index_list) == 0:
            print("没有相匹配的作者信息")
            return aa
        for i in self.index_list:
            mm = self.author_list[i] + ' 机构:' + author_organ_list[i]
            if author_organ_list[i][0:3] not in quchong_list:
                self.authors_dict[j] = mm
                j += 1
            quchong_list.append(author_organ_list[i][0:3])

        for i in self.index_list:
            mm = self.author_list[i] + ' 机构:' + author_organ_list[i]
            if author_organ_list[i][0:3] not in quchong_list2:
                self.author_dict[j] = mm
                if self.organ in mm:
                    self.author_index = j
                    break
                else:
                    j += 1
            quchong_list2.append(author_organ_list[i][0:3])
        return self.author()

    def author(self):
        if self.organ == '':
            print(self.authors_dict)
            print("请输入作者的所属机构")
            return "qqqq"
        n = int(self.author_index)
        haha = str(self.author_dict[n]).split(' ')
        name = str(haha[0])
        ha = str(haha[1])
        organ = str(ha.split(':')[1])
        paper = []
        parter = []
        jounal = []
        keywords = []
        paper_list = list(self.papers_author['title'])
        author_list = list(self.papers_author['first_authors'])
        authors_list = list(self.papers_author['authors'])
        parter_list = list(self.papers_author['parter_authors'])
        authors_organ_list = list(self.papers_author['organ'])
        keywords_list = list(self.papers_author['keywords'])
        jounal_list = list(self.papers_author['Journal'])
        author_index = [idx for idx, author in enumerate(authors_list) if name in author.split(';')]
        organ_index = [idx for idx, organs in enumerate(authors_organ_list) if organ in organs.split(';')]
        res_list = self.commonElement(author_index, organ_index)
        for j in res_list:
            a = paper_list[j]
            e = jounal_list[j]
            jounal.append(e)
            paper.append(a)
        tonghang = []
        qikan = []
        for j in res_list:
            e = jounal_list[j]
            if e not in qikan:
                qikan.append(e)

        for j in res_list:
            d = re.split(';;|;', keywords_list[j])
            for ii in d:
                if ii != '' and len(ii) < 10:
                    keywords.append(ii)
        # 同行、合作者
        for j in res_list:
            b = str(parter_list[j]).split(" ")
            for ii in b:
                if ii != 'none' and ii != name and ii != '' and ii != ']':
                    ii = ii.strip("\'").strip(',').strip('[')
                    parter.append(ii.strip("'"))
                    if ii not in tonghang:
                        tonghang.append(ii.strip(',').strip("'").strip('['))
        mmm_list = []
        for i in range(len(keywords_list)):
            if keywords[0] in keywords_list[i]:
                mmm_list.append(i)
        for j in mmm_list:
            ii = author_list[j]
            if ii != '' and ii != name:
                tonghang.append(ii.strip(',').strip("'").strip('['))
        if len(tonghang) >= 6:
            tonghang = tonghang[-6:-1]
        else:
            tonghang = tonghang
        if "['']" in tonghang:
            tonghang.remove("['']")
        if ']' in tonghang:
            tonghang.remove(']')
        # 关键词
        keyword = self.all_list(keywords)
        keyword_s = []
        for j in keyword.keys():
            keyword_s.append(j)
        if len(keyword_s) > 10:
            keyword_s = keyword_s[0:10]
        else:
            keyword_s = keyword_s
        # 期刊
        jounals = self.all_list(jounal)
        a_list = []
        mm = sorted(jounals.items(), key=lambda x: x[1], reverse=True)
        for i in range(len(jounals)):
            a_list.append(mm[i][0])
        if len(a_list) > 3:
            a_list = a_list[0:3]
        else:
            a_list = a_list
        parters_index = self.all_list(parter)
        if ']' in parters_index:
            del parters_index[']']
        if "['']" in parters_index:
            del parters_index["['']"]
        mmmm = sorted(parters_index.items(), key=lambda x: x[1], reverse=True)
        result = {'Author': self.name, 'Organ': organ, 'Partner': mmmm,
                  'Journal': a_list, 'Papers': paper, 'Keywords': keyword_s, 'Peer': tonghang}
        return result
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
class pre_word_count():
    def __init__(self, data_path, model_path, hotword_num, hotyear_num):
        self.data = pd.read_json(data_path)
        self.data['authors_seg'] = self.data.apply(lambda r: r['P_Red1'] + r['P_Red3'], axis=1)
        self.data['year'] = self.data['P_year'].apply(lambda x: x[:4])
        self.model_path = model_path
        self.hotword_num = hotword_num
        self.hotyear_num = hotyear_num

    # 统计出现在句子中的词频
    def tf(self, word, sentence):
        c = 0
        s = sentence.split()
        for i in s:
            if word == i:
                c += 1
        return c

    # 统计非重复的所有作者
    def tongjiau(self, li):
        authors = []
        for i in li:
            try:
                authors += i.split()
            except:
                continue
        return list(set(authors))

    # 判断新作者是否在论文中提及热点词
    def mention(self, authors, word, sentences, senten_auth):
        i = 0
        for a in authors:
            for j in range(len(sentences)):
                try:
                    if len(list(set(a).intersection(set(senten_auth[j])))) > 0 and word in sentences[j]:
                        i += 1
                except:
                    continue
        return i

    # 统计某个热点词在某年的不同期刊的提及数
    def multi_journal(self, word, sentences, journals):
        men_journal = []
        for j in range(len(sentences)):
            try:
                if word in sentences[j]:
                    men_journal.append(journals[j])
            except:
                continue
        s = set(men_journal)
        if None in s:
            s.remove(None)
        return len(s)

    # 抽取词频特征
    def feature(self, word, year_df):
        abstract_tf = []
        keyword_tf = []
        for y in list(year_df['seg_abstract']):
            abstract_tf.append(self.tf(word, y))
        for i in list(year_df['seg_keywords']):
            keyword_tf.append(self.tf(word, i))
        return abstract_tf, keyword_tf

    # 计算list的平均数

    def pingjun(self, L):
        if len(L) == 0:
            return 0
        else:
            s = 0
            for i in L:
                s += i

            return int(s / len(L))

    # 获取热点词
    def get_hotwords(self, hotword_num):
        guanjian = ' '.join(list(self.data['P_Keyword_seg'])).split()
        # textrank方法获取关键词

        # merge=' '.join(guanjian)
        # tr4w = TextRank4Keyword()
        # tr4w.analyze(text=merge, window=4)
        # pred_words=[]
        # for item in tr4w.get_keywords(hotword_num, word_min_len=2):
        # pred_words.append(item.word)
        # 根据词频排序获取关键词
        result = Counter(guanjian)
        d = sorted(result.items(), key=lambda x: x[1], reverse=True)
        pred_words = list(map(lambda x: x[0], d))[:hotword_num]
        return pred_words

    # 统计每年发表的论文数
    def year_paper_num(self, ):
        year_num = []
        years = list(set(self.data['year']))
        group_df = self.data.groupby(by=['year'])
        # years.remove(None)
        years = sorted([int(i) for i in years])
        for y in years:
            try:
                year_num.append(len(group_df.get_group(str(y))))
            except:
                continue
        return group_df, years, year_num

    # 按年份将摘要整合
    def groupby_year(self, group_df, years, year_num):
        years_df = pd.DataFrame()
        years_df['year'] = years
        seg, key, tit, jou, authors = [], [], [], [], []

        for i in years:
            seg.append(' '.join(group_df.get_group(str(i))['P_Summary_seg']))
            k = group_df.get_group(str(i))['P_Keyword_seg']
            key.append(' '.join(k))
            tit.append(' '.join(group_df.get_group(str(i))['P_Title_seg']))
            jou.append(' '.join(group_df.get_group(str(i))['P_Publication']))
            authors.append(self.tongjiau(group_df.get_group(str(i))['authors_seg']))
        years_df['seg_abstract'] = seg
        years_df['seg_keywords'] = key
        years_df['seg_title'] = tit
        years_df['seg_authors'] = authors
        years_df['Jounal'] = jou
        years_df['paper_num'] = year_num
        return years_df

    # 统计每年的新作者
    def year_new_author(self, years_df):
        authors = list(years_df['seg_authors'])
        new_author = [authors[0]]
        for i in range(1, len(years_df)):
            new = []
            for j in authors[i]:
                if j not in authors[i - 1]:
                    new.append(j)
            new_author.append(new)
        return new_author

    # 统计每年新作者提及数
    def year_new_author_mention(self, pred_words, new_author, years):
        newauthor_mens = []
        group_df = self.data.groupby(by=['year'])
        for word in pred_words:
            for i in range(len(new_author)):
                newauthor_men = self.mention(new_author, word, list(group_df.get_group(str(years[i]))['P_Summary']), \
                                             list(map(lambda x: x.split(),
                                                      list(group_df.get_group(str(years[i]))['authors_seg']))))
                newauthor_mens.append(newauthor_men)
        return newauthor_mens

    # 统计跨学科提及数
    def multi_sub(self, pred_words, years):
        multi_mens = []
        group_df = self.data.groupby(by=['year'])
        for word in pred_words:
            for i in range(len(years)):
                journals = self.multi_journal(word, list(group_df.get_group(str(years[i]))['P_Summary']), \
                                              list(group_df.get_group(str(years[i]))['P_Publication']))
                multi_mens.append(journals)
        return multi_mens

    # 统计每个词初次在关键词中出现的年份
    def first_year(self, pred_words, years_df):
        f_year = []
        pp = []
        for w in range(len(pred_words)):
            for y, i in zip(list(years_df['year']), list(years_df['seg_keywords'])):
                if pred_words[w] in i:
                    f_year.append(y)
                    pp.append(w)
                    break
        return f_year

    def sub_df(self, pp, years_df, hot_year_num, hot_word_num, out_year, n_year):
        year_num = len(years_df)
        # 判断year组合是否有多余年份
        pp_first = pp['word']
        pp_first_line = []
        for hy in range(hot_word_num):
            for hyy in range(n_year):
                pp_first_line.append(pp_first[hy*year_num])
        pp_first_line = pd.DataFrame(pp_first_line)
        pp_first_line.columns = ['word']
        ppp = pp.drop(columns=['word'])
        sub_dfq = {}
        hot_nyear = []
        for i in range(hot_word_num):
            sub_dfq[i] = ppp.iloc[year_num * i+out_year: year_num * (i+1)]
            sub_df_of_one = {}
            for j in range(n_year):
                sub_df_of_one[j] = sub_dfq[i].iloc[hot_year_num * j: hot_year_num * (j+1)]
                sub_df_of_one[j].loc['Row'] = sub_df_of_one[j].apply(lambda x: x.sum())
                hot_nyear.append(sub_df_of_one[j].loc['Row'])
        df_hot_nyear = pd.DataFrame(hot_nyear)
        df_hot_nyear.reset_index(inplace=True)
        df_hot_nyear["word"]=pp_first_line
        df_hot_nyear=df_hot_nyear[["word","f1","f2","f3","f4","f6"]]
        return df_hot_nyear

    # 获取每个词18年的特征
    def get_feature(self, years_df, pred_words, f_year, newauthor_mens, multi_mens):
        year_num = len(years_df)
        pp = pd.DataFrame()
        for i in range(len(pred_words)):
            word = pred_words[i]
            fyear = f_year[i]
            exist_year = []
            for n, f in zip(list(years_df['year']), [fyear] * year_num):
                e = n - f
                if e < 0:
                    exist_year.append(0)
                else:
                    exist_year.append(e)
            dd = pd.DataFrame()
            dd['word'] = [word] * year_num
            dd['f1'] = self.feature(word, years_df)[0]
            dd['f2'] = self.feature(word, years_df)[1]
            dd['f3'] = exist_year
            dd['f4'] = newauthor_mens[i * year_num:i * year_num + (year_num - 1)] + \
                       [self.pingjun(newauthor_mens[i * year_num + (year_num - 1) - 3:i * year_num + (year_num - 1)])]

            dd['f6'] = multi_mens[i * year_num:i * year_num + (year_num - 1)] + \
                       [self.pingjun(multi_mens[i * year_num + (year_num - 1) - 3:i * year_num + (year_num - 1)])]

            if i == 0:
                pp = dd
            else:
                pp = pp.append(dd)
        pp = pp.reset_index(drop=True)
        return pp

    # 预测某个词的词频
    def zhidingword(self, search_word, test_data, model):
        pre_data = test_data.loc[test_data.word == search_word].iloc[:, 1:]
        result = model.predict(pre_data)
        return int(result)

    # 返回结果
    def pre_result(self):
        model_GBR = joblib.load(self.model_path)
        pred_words = self.get_hotwords(self.hotword_num)
        years = self.year_paper_num()[1]
        group_df = self.year_paper_num()[0]
        year_num = self.year_paper_num()[2]
        years_df = self.groupby_year(group_df, years, year_num)
        f_year = self.first_year(pred_words, years_df)
        new_author = self.year_new_author(years_df)
        newauthor_mens = self.year_new_author_mention(pred_words, new_author, years)
        multi_mens = self.multi_sub(pred_words, years)
        pp = self.get_feature(years_df, pred_words, f_year, newauthor_mens, multi_mens)

        # out_year = len(years_df) % self.hotyear_num  ## 年余数
        n_year = len(years_df) // self.hotyear_num  ## 多少组年
        if n_year == 0:
            out_year = len(years_df) % 1  ## 年余数
            n_year = len(years_df) // 1  ## 多少组年
            hotyear_numqq = 1
        else:
            out_year = len(years_df) % self.hotyear_num  ## 年余数
            n_year = len(years_df) // self.hotyear_num  ## 多少组年
            hotyear_numqq = self.hotyear_num

        hot_nyear = self.sub_df(pp, years_df, hotyear_numqq, self.hotword_num, out_year, n_year)

        num = list(range(n_year - 1, n_year * self.hotword_num, n_year))
        test_data = hot_nyear.iloc[num]
        all_wordtf = []
        for word in pred_words:
            result = self.zhidingword(word, test_data, model_GBR)
            search_ke = list(hot_nyear.loc[hot_nyear.word == word]['f1']) + [result]
            all_wordtf.extend(search_ke)
        result = pd.DataFrame()
        next_year = years[-1] + hotyear_numqq
        years_nhotyear = []
        for i in range(n_year):
            years_nhotyear.append(years[i*hotyear_numqq+out_year+hotyear_numqq-1])
        years_nhotyear.append(next_year)
        wword_pred=[]
        for w in pred_words:
            for ww in years_nhotyear:
                wword_pred.append(w)
        result['word'] = wword_pred
        result['year'] = years_nhotyear * len(pred_words)
        result['count'] = all_wordtf
        word_next_year = result.loc[result['year'] == next_year]
        word_next_year_new = word_next_year.sort_values(by="count", ascending=False)
        Top_hot_word = list(word_next_year_new.loc[result['year'] == next_year]['word'])

        hot_year = list(years_nhotyear)
        freq_word = []
        freq = []
        freqs = []
        for www in Top_hot_word:
            freq_word.append(www)
            freq = list(result.loc[result['word'] == www].sort_values(by="year", ascending=True)['count'])
            freqs.append(freq)
        draw = pd.DataFrame()
        draw["draw_word"] = freq_word
        draw["draw_freqs"] = freqs
        draw_dict = [{'draw_word': i[1], 'draw_freqs': i[2]} for i in draw.itertuples()]
        #         draw_json=draw.to_json(orient='index')
        altm = {'topnn': Top_hot_word, 'year': hot_year, 'draw': draw_dict}
        return altm
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def tongjiau(li):
    authors = []
    for i in li:
        try:
            authors += i
        except:
            continue
    return list(set(authors))


def savepickle(name, data):
    output = open('%s.pkl' % (name), 'wb')
    pkl.dump(data, output)
    output.close()


def loadpickle(name):
    pkl_file = open('%s.pkl' % (name), 'rb')
    data1 = pkl.load(pkl_file)
    return data1


def get_data(key, from_date, to_date, volume):
    es4Paper = esPaperRetrieval(host='10.8.128.205', port=49200)
    data = es4Paper.Retrieval(titleQuery=key, kwordQuery=key, summaryQuery=key, pubQuery='',
                              fromDate=from_date, toDate=to_date, volume=volume)
    data = pd.DataFrame(data)[["P_Author", "P_Publication", "P_Summary_seg", "P_Keyword", "P_year"]]
    data.columns = ["authors", "Jounal", "seg_abstract", "keywords", "year"]
    data.year = [i[0:4] for i in data.year]

    group_df = data.groupby(by=['year'])
    # 统计每年的论文数量
    years = list(set(data['year']))
    # years.remove(None)
    years = sorted([int(i) for i in years])
    year_num = []
    for y in years:
        try:
            year_num.append(len(group_df.get_group(str(y))))
        except:
            year_num.append(0)

    # 根据年份将文献信息整合并保存

    years_df = pd.DataFrame()
    years_df['year'] = years
    seg = []
    key = []
    jou = []
    authors = []
    for i in years:
        seg.append(' '.join(list(group_df.get_group(str(i))['seg_abstract'])))
        k = list(group_df.get_group(str(i))['keywords'])
        if None in k:
            while None in k:
                k.remove(None)
            key.append(' '.join(k))
        else:
            key.append(' '.join(k))
        jou.append(' '.join(list(group_df.get_group(str(i))['Jounal'])))
        authors.append(' '.join(tongjiau(list(group_df.get_group(str(i))['authors']))).split())
    years_df['seg_abstract'] = seg
    years_df['keywords'] = key
    years_df['authors'] = authors
    years_df['Jounal'] = jou
    years_df['paper_num'] = year_num
    savepickle('year_df', years_df)
    return years_df


def print_top_words_list(model, feature_names, n_top_words):
    result = []
    for topic_idx, topic in enumerate(model.components_):
        result += [[feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]]
    return result


def tf_lda_list(n_topic, n_top_words, max_feature, data):
    vectorizer = CountVectorizer(max_features=max_feature)
    X = vectorizer.fit_transform(data)
    X = X.toarray()
    # 主题词个数
    n_top_words = n_top_words
    n_samples = X.shape[0]
    n_features = X.shape[1]
    print("Fitting LDA models with tf features, "
          "n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    lda = LatentDirichletAllocation(n_components=n_topic, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)

    # print("\nTopics in LDA model:")
    lda.fit(X)
    tf_feature_names = vectorizer.get_feature_names()
    return print_top_words_list(lda, tf_feature_names, n_top_words)


# In[55]:


def zhutifenxi(key, n_topic=3, n_top_words=30, year_from=2010, year_to=2018, volume=500):
    years_df = get_data(key, str(year_from - 1) + "-12-31", str(year_to) + "-12-31", volume)
    max_feature = 2000
    result = []
    result += tf_lda_list(n_topic, n_top_words, max_feature,
                          years_df.loc[years_df.year.isin(range(year_from, year_to + 1))]['seg_abstract'])
    value = [[100] * n_top_words for i in range(n_topic)]
    key = [[("name", "value")] * n_top_words for i in range(n_topic)]

    return [[dict(zip(t[0], t[1])) for t in k] for k in
            [list(zip(j[0], j[1])) for j in list(zip(key, [list(zip(i[0], i[1])) for i in list(zip(result, value))]))]]


def build_tfidf(max_feature, data):
    #     vectorizer=CountVectorizer()
    tfidf = TfidfVectorizer(max_features=max_feature)
    f_tfidf = tfidf.fit_transform(data)
    word = tfidf.get_feature_names()
    f_tfidf = f_tfidf.toarray()
    return f_tfidf, word


def print_top_words(model, feature_names, n_top_words):
    result_str = ''
    for topic_idx, topic in enumerate(model.components_):
        result_str += "Topic #%d:\n" % topic_idx
        result_str += " ".join([feature_names[i]
                                for i in topic.argsort()[:-n_top_words - 1:-1]]) + '\n'
    return result_str


def tfidf_lda(n_topic, n_top_words, max_feature, data):
    tfidf = TfidfVectorizer(max_features=max_feature)
    f_tfidf = tfidf.fit_transform(data)
    f_tfidf = f_tfidf.toarray()
    # 主题词个数
    n_top_words = n_top_words
    n_samples = f_tfidf.shape[0]
    n_features = f_tfidf.shape[1]

    lda = LatentDirichletAllocation(n_components=n_topic, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)

    #     print("\nTopics in LDA model:")
    lda.fit(f_tfidf)
    tf_feature_names = tfidf.get_feature_names()
    return print_top_words_list(lda, tf_feature_names, n_top_words)


def youxujulei(key, n_class=4, n_topic=1, n_top_topic=30):
    years_df = get_data(key, "2010-12-31", "2019-12-31", 1000)
    # 设置特征维度
    max_feature = 2000
    # 根据年份建立tfidf表
    year_tfidf = build_tfidf(max_feature, years_df['seg_abstract'])
    # 根据年份建立tf表
    # year_tf=build_tf(max_feature,years_df['seg_abstract'])

    # 有序聚类

    # 聚类数量
    lst = get_one_essay_trace(year_tfidf[0], n_class)
    split_result = observe_divided(lst[0], years_df['year'])

    split_df = pd.DataFrame()
    abstract_agg = []
    abstracts = []
    for i in split_result:
        tmp = []
        for y in i:
            tmp.append(list(years_df.loc[years_df.year == y]['seg_abstract'])[0])
        abstract_agg.append(' '.join(tmp))
        abstracts.append(tmp)
    split_df['class'] = split_result
    split_df['merge_abstract'] = abstract_agg
    split_df['abstracts'] = abstracts

    # 根据tf计算LDA主题词
    # 算法 类名/第i类
    result_str = []
    keys = ["pub_id", "title", "publicationDate", "firstApplicant"]
    for i in range(n_class):
        t = 1
        for j in tf_lda_list(n_topic, n_top_topic, max_feature, split_df.loc[i, 'abstracts']):
            result_str += [dict(zip(keys, ["tf-lda", str(split_df['class'][i]) + "年", "主题" + str(t), " ".join(j)]))]
            t += 1

    # 根据tfidf计算LDA主题词
    for i in range(n_class):
        t = 1
        for j in tfidf_lda(n_topic, n_top_topic, max_feature, split_df.loc[i, 'abstracts']):
            result_str += [dict(zip(keys, ["tfidf-lda", "第" + str(i + 1) + "类", "主题" + str(t), " ".join(j)]))]
            t += 1

    return result_str
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def distance(a, b):
    #  计算两个vector的距离，逐项差的平方和，不开根号
    if len(a) != len(b):
        print('wrong')
        return 0
    else:
        temp = a - b
        temp = temp ** 2
        res = sum(temp)
        return res
def get_dist_mat(feat):
    #  输入为特征矩阵，返回一个相似度矩阵（行数乘行数）
    print(type(feat))
    length=feat.shape[0]#样本数量
    wide = feat.shape[1]#特征向量长度
    res = np.zeros(shape=(length, length))
    ave = np.zeros(shape=(length, length, wide))

    for i in range(0, length):
        for j in range(i+1, length):
            ave[i, j] = np.mean(feat[i: j+1], axis=0)
            # print('i = %d, j = %d' % (i, j))

    for i in range(0, length):
        for j in range(i+1, length):
            ave_one = ave[i, j]
            dist_lst = [distance(ave_one, one) for one in feat[i: j+1]]
            res[i, j] = sum(dist_lst)
            # print('i = %d, j = %d' % (i, j))
    return res

def get_class_devide(dist_mat, class_num):
    #  利用递推公式计算分成class_num类别的分割方法
    length = len(dist_mat)
    divided_point = np.zeros(shape=(length, class_num+1))  # 存切割点
    diveded_dist = np.zeros(shape=(length, class_num+1))  # 存切割最小距离

    for i in range(1, length):  # 分成两类
        dist_lst = [dist_mat[0, k] + dist_mat[k+1, i] for k in range(0, i)]
        divided_point[i, 2] = np.argmin(dist_lst)
        diveded_dist[i, 2] = np.min(dist_lst)

    for classes in range(3, class_num+1):   # 分成多类
        for i in range(classes-1, length):
            dist_lst = [diveded_dist[k, classes-1] + dist_mat[k+1, i] for k in range(classes-2, i)]  # 从n-1类到n类
            divided_point[i, classes] = np.argmin(dist_lst) + classes - 2
            diveded_dist[i, classes] = np.min(dist_lst)
    return diveded_dist, divided_point

def get_trace(trace_mat, class_num):
    #  获得切割方法list，
    #  trace_mat就是get_class_devide函数返回的divided_point，分割点记录矩阵
    lst = []
    length = len(trace_mat)  #length=38
    pre = length - 1   #pre=37
    for i in range(class_num, 1, -1):
        temp = int(trace_mat[pre, i])
        pre = temp
        lst.append(temp)
    lst.reverse()
    return lst
def get_one_essay_trace(feature,phrase_num):
    #  整合调用 有序聚类方法
    #  调用就可以获得一个文件的分割结果并存入本地

    time_start = time.time()
    dist_mat = get_dist_mat(feature)
    phrase_num = phrase_num
    min_dist_mat, min_trace_mat = get_class_devide(dist_mat, phrase_num)
    # print(min_trace_mat)

    lst = get_trace(min_trace_mat, phrase_num)
    lst.append(len(feature))
    print('划分结果为：',end='')
    print(lst)
    time_end = time.time()
    print('cost time', time_end - time_start)
    return lst,min_dist_mat,min_trace_mat
def observe_divided(lst, sents):
    # 展示分割结果
    split_result=[]
    start = 0
    for end in lst:
        cla_lst=[]
        if end==len(sents):
            end=end-1
        for i in range(start, end+1):
            cla_lst.append(sents[i])
        start = end + 1
        split_result.append(cla_lst)
    return split_result


In [6]:

esPaperObj = esPaperRetrieval(host='10.8.128.205',port=49200)
docs = esPaperObj.Retrieval(titleQuery='',kwordQuery='',summaryQuery='医',pubQuery='', fromDate='2016-01-01',toDate='2019-01-01',volume=1000)

hotPointObj = pre_word_count(json.dumps(docs), '../data/paper_data/paper_hotpoint.model',15, 2)
h=hotPointObj.pre_result()


/home/lawbda/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:445: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'topnn': ['中医药', '中医医院', '中医', '医院', '医疗设备', '影响因素', '公立医院', '医患关系', '医联体', '管理', '满意度', '中医药法', '分级诊疗', '对策', '实践'], 'year': [2018, 2020], 'draw': [{'draw_word': '中医药', 'draw_freqs': [213, 365]}, {'draw_word': '中医医院', 'draw_freqs': [236, 303]}, {'draw_word': '中医', 'draw_freqs': [472, 274]}, {'draw_word': '医院', 'draw_freqs': [747, 240]}, {'draw_word': '医疗设备', 'draw_freqs': [70, 129]}, {'draw_word': '影响因素', 'draw_freqs': [74, 129]}, {'draw_word': '公立医院', 'draw_freqs': [95, 110]}, {'draw_word': '医患关系', 'draw_freqs': [183, 96]}, {'draw_word': '医联体', 'draw_freqs': [122, 89]}, {'draw_word': '管理', 'draw_freqs': [97, 61]}, {'draw_word': '满意度', 'draw_freqs': [60, 35]}, {'draw_word': '中医药法', 'draw_freqs': [48, 21]}, {'draw_word': '分级诊疗', 'draw_freqs': [30, 21]}, {'draw_word': '对策', 'draw_freqs': [38, 20]}, {'draw_word': '实践', 'draw_freqs': [45, 19]}]}


In [7]:
h

{'topnn': ['中医药',
  '中医医院',
  '中医',
  '医院',
  '医疗设备',
  '影响因素',
  '公立医院',
  '医患关系',
  '医联体',
  '管理',
  '满意度',
  '中医药法',
  '分级诊疗',
  '对策',
  '实践'],
 'year': [2018, 2020],
 'draw': [{'draw_word': '中医药', 'draw_freqs': [213, 365]},
  {'draw_word': '中医医院', 'draw_freqs': [236, 303]},
  {'draw_word': '中医', 'draw_freqs': [472, 274]},
  {'draw_word': '医院', 'draw_freqs': [747, 240]},
  {'draw_word': '医疗设备', 'draw_freqs': [70, 129]},
  {'draw_word': '影响因素', 'draw_freqs': [74, 129]},
  {'draw_word': '公立医院', 'draw_freqs': [95, 110]},
  {'draw_word': '医患关系', 'draw_freqs': [183, 96]},
  {'draw_word': '医联体', 'draw_freqs': [122, 89]},
  {'draw_word': '管理', 'draw_freqs': [97, 61]},
  {'draw_word': '满意度', 'draw_freqs': [60, 35]},
  {'draw_word': '中医药法', 'draw_freqs': [48, 21]},
  {'draw_word': '分级诊疗', 'draw_freqs': [30, 21]},
  {'draw_word': '对策', 'draw_freqs': [38, 20]},
  {'draw_word': '实践', 'draw_freqs': [45, 19]}]}